In [1]:
import sys
import os
%matplotlib inline

In [2]:
os.getcwd()

'c:\\Users\\gabi1\\git\\ACTC_MD_ML\\gaby_notebooks'

In [3]:
os.chdir('..\\')
os.getcwd()

'c:\\Users\\gabi1\\git\\ACTC_MD_ML'

In [4]:
from rdkit.Chem.Draw import rdMolDraw2D
from src.trainers import Trainers
from src.polymerSMILES import polymers
from src.dataProcessing import * #DataPreprocessing #computeTanimotoSimilarity #* #data_load, computeTanimotoSimilarity, get_dissimilarity_map, featurizer, normalizedata
from src.solver import *

# Data preprocessing part

### Load polymer smiles

In [5]:
# load smiles
polymers_dict = polymers.get_data()

### Implement datapreprocessing class

In [6]:
# data preprocessing toolbox
preprocess = DataPreprocessing()

### Loading the data

- data1 will be use for activity

In [7]:
data1 = preprocess.data_load(filename ='1-0-hgk-MD-ActCoeff-processed_Activity.csv')
data1.head()

,#,Name of the polymer,CounterIon,Co-Ion,salt,Water-per-ion,concentration of salt (M),gr_minima (Ion_H2O),gr_peak_position (Ion_H2O),gr_peak_height (Ion_H2O),Nr_(Ion_H2O),gr_minima (CG_H2O),gr_peak_position (CG_H2O),gr_peak_hight (CG_H2O),Nr_(CG_H2O),gr_minima (CG_Ion),gr_peak_position (CG_Ion),gr_peak_height (CG_Ion),Nr_(CG_Ion),Experimental activity co.
0,1,PSbNMP_RCE,I-,K+,KI,6,0.02,4.45,3.65,6.50,4.70,5.50,3.55,2.30,6.10,6.15,4.15,5.80,1.90,0.300000
1,2,PSbNMP_RCE,I-,K+,KI,6,0.20,4.45,3.65,6.30,4.60,5.50,3.55,1.90,5.00,6.15,4.15,5.80,2.00,0.319000
2,3,PSbNMP_RCE,I-,K+,KI,6,0.50,4.45,3.65,6.20,4.60,5.50,3.55,1.80,4.50,6.15,4.15,5.60,2.20,0.476000
3,4,PSbNMP_RCE,I-,K+,KI,6,1.00,4.55,3.65,6.00,4.70,5.00,3.55,1.50,3.00,6.15,4.15,5.10,2.50,0.663000
4,5,PVAS_RCE,Na+,Cl-,NaCl,113,0.02,3.15,2.35,8.55,5.38,3.37,2.76,1.83,2.98,3.06,2.35,6.71,0.13,0.280899


- data2 will be use for solvation

In [8]:
data2 = preprocess.data_load(filename = "1.0-hkg-MD-ActCoeff-processed.csv")
data2.head()

,#,Name of the polymer,CounterIon,Co-Ion,salt,Water-per-ion,concentration_of_salt_M,gr_minima_Ion_H2O,gr_peak_position_Ion_H2O,gr_peak_height_Ion_H2O,Nr_Ion_H2O,gr_minima_CG_H2O,gr_peak_position_CG_H2O,gr_peak_height_CG_H2O,Nr_CG_H2O_,gr_minima_CG_Ion,gr_peak_position_CG_Ion,gr_peak_height_CG_Ion,Nr_CG_Ion,Exp_act_coeff
0,1,PSbNMP_RCE,I-,K+,KI,6,0.00,4.45,3.65,6.5,4.7,5.4,3.55,2.16,5.3,6.25,4.15,5.6,1.9,-
1,2,PSbNMP_RCE,I-,K+,KI,6,0.02,4.45,3.65,6.5,4.7,5.5,3.55,2.30,6.1,6.15,4.15,5.8,1.9,0.300
2,3,PSbNMP_RCE,I-,K+,KI,6,0.20,4.45,3.65,6.3,4.6,5.5,3.55,1.90,5.0,6.15,4.15,5.8,2.0,0.319
3,4,PSbNMP_RCE,I-,K+,KI,6,0.50,4.45,3.65,6.2,4.6,5.5,3.55,1.80,4.5,6.15,4.15,5.6,2.2,0.476
4,5,PSbNMP_RCE,I-,K+,KI,6,1.00,4.55,3.65,6.0,4.7,5.0,3.55,1.50,3.0,6.15,4.15,5.1,2.5,0.663


### Featurizer the data 

- this is going to define X and y depending on the descr.

#### For activity

In [9]:
## assign X1, y1 when descr = activity 
(X1, y1, MFprints) = preprocess.featurizer(data1, polymers_dict,  'activity') # descr = 'solvation' or 'activity'

In [10]:
#since the descr == activity X should be reverse_fingerprint + Ohe_df and data_mm(mixcared)
X1

,mfp_0,mfp_1,mfp_2,mfp_3,mfp_4,mfp_5,mfp_6,mfp_7,mfp_8,mfp_9,...,E,F,G,H,I,J,K,L,M,N
0,0,1,0,1,0,1,1,0,0,0,...,-0.940850,-0.022222,0.614035,-0.028571,-0.324042,-0.465909,0.937322,0.525424,-0.865209,0.514170
1,0,1,0,1,0,1,1,0,0,0,...,-0.953173,-0.066667,0.614035,-0.028571,-0.602787,-0.590909,0.937322,0.525424,-0.865209,0.595142
2,0,1,0,1,0,1,1,0,0,0,...,-0.959335,-0.066667,0.614035,-0.028571,-0.672474,-0.647727,0.937322,0.525424,-0.872495,0.757085
3,0,1,0,1,0,1,1,0,0,0,...,-0.971657,-0.022222,0.263158,-0.028571,-0.881533,-0.818182,0.937322,0.525424,-0.890710,1.000000
4,0,1,0,1,0,1,1,0,0,0,...,-0.814541,0.280000,-0.880702,-0.931429,-0.651568,-0.820455,-0.823362,-0.694915,-0.832058,-0.919028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1,1,1,0,1,1,0,0,1,0,...,-0.736907,-0.057778,-0.838596,-0.942857,-0.749129,-0.929545,-0.851852,-0.694915,-0.795264,-0.910931
76,1,1,1,0,1,1,0,0,1,0,...,-0.815773,-0.315556,-0.719298,-0.942857,-0.909408,-0.886364,-0.829060,-0.694915,-0.543169,-0.773279
77,1,1,1,0,1,1,0,0,1,0,...,-0.743685,-0.066667,-0.754386,-0.942857,-0.756098,-0.894318,-0.829060,-0.694915,-0.774135,-0.878543
78,1,1,1,0,1,1,0,0,1,0,...,-0.749846,-0.102222,-0.754386,-0.942857,-0.770035,-0.901136,-0.823362,-0.694915,-0.765027,-0.870445


In [11]:
#y should be the activity coef column only
y1

0     0.300000
1     0.319000
2     0.476000
3     0.663000
4     0.280899
        ...   
75    0.600412
76    0.685773
77    0.667216
78    0.704330
79    0.819381
Name: Experimental activity co., Length: 80, dtype: float64

#### For solvation

In [12]:
## assign X2, y2 when descr = solvation 
(X2, y2, MFprints) = preprocess.featurizer(data2, polymers_dict,  'solvation') # descr = 'solvation' or 'activity'

In [13]:
# X2 contains reverse_fingerprint + Ohe_df + water per ion and concentration of salt
X2

,mfp_0,mfp_1,mfp_2,mfp_3,mfp_4,mfp_5,mfp_6,mfp_7,mfp_8,mfp_9,...,ohe_3,ohe_4,ohe_5,ohe_6,ohe_7,ohe_8,ohe_9,ohe_10,Water-per-ion,concentration_of_salt_M
0,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.00
1,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.02
2,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.20
3,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,0.50
4,0,1,1,1,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,1,1,0,1,1,1,0,0,1,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,24,0.01
88,1,1,0,1,1,1,0,0,1,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,24,0.03
89,1,1,0,1,1,1,0,0,1,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,24,0.10
90,1,1,0,1,1,1,0,0,1,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,24,0.30


In [14]:
# y2 contains the rest of the parameter in the data2
y2

,gr_minima_Ion_H2O,gr_peak_position_Ion_H2O,gr_peak_height_Ion_H2O,Nr_Ion_H2O,gr_minima_CG_H2O,gr_peak_position_CG_H2O,gr_peak_height_CG_H2O,Nr_CG_H2O_,gr_minima_CG_Ion,gr_peak_position_CG_Ion,gr_peak_height_CG_Ion,Nr_CG_Ion
0,4.45,3.65,6.50,4.70,5.40,3.55,2.16,5.30,6.25,4.15,5.60,1.90
1,4.45,3.65,6.50,4.70,5.50,3.55,2.30,6.10,6.15,4.15,5.80,1.90
2,4.45,3.65,6.30,4.60,5.50,3.55,1.90,5.00,6.15,4.15,5.80,2.00
3,4.45,3.65,6.20,4.60,5.50,3.55,1.80,4.50,6.15,4.15,5.60,2.20
4,4.55,3.65,6.00,4.70,5.00,3.55,1.50,3.00,6.15,4.15,5.10,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...
87,3.18,2.35,9.81,4.62,3.43,2.75,1.69,2.02,3.01,2.35,7.72,0.14
88,3.16,2.35,8.53,4.04,3.60,2.75,1.46,2.40,3.05,2.35,14.64,0.31
89,3.17,2.35,9.70,4.60,3.55,2.75,1.68,2.33,3.05,2.35,8.30,0.18
90,3.15,2.35,9.60,4.52,3.55,2.75,1.66,2.27,3.06,2.35,8.55,0.19


### Split and normailize data

#### Activity

In [15]:
(data_activity_scaler, X_train1, X_test1, y_train1, y_test1) = preprocess.normalizedata(X1, y1, splitRatio=0.2, state = 48, transform = True, property = None)

#### Solvation

In [16]:
(data_solvation_scaler, X_train2, X_test2, y_train2, y_test2) = preprocess.normalizedata(X2, y2, splitRatio=0.2, state = 48, transform = True, property = 'Y')

### PCA transformation for A1 and A2

Reminder the the input vestors:
- A0 represents all data with no transformation
- A1 PCA applied only to the polymer fingerprint
- A2 PCE applied to all input features

#### Activity

##### A1 (MDFP represent the polymer fingerprint input)

In [17]:
pca_X1_train_MDFP, pca_X1_test_MDFP = preprocess.convertInputsPCA(X_train1, X_test1, MFprints, n_comp = 15, type = 'MDFP')

##### A2 (ALL represent all input)

In [18]:
pca_X1_train_ALL, pca_X1_test_ALL = preprocess.convertInputsPCA(X_train1, X_test1, MFprints, n_comp = 15, type = 'All')

#### Solvation

##### A1

In [19]:
pca_X2_train_MDFP, pca_X2_test_MDFP = preprocess.convertInputsPCA(X_train2, X_test2, MFprints, n_comp=10, type = 'MDFP')

##### A2

In [20]:
pca_X2_train_ALL, pca_X2_test_ALL = preprocess.convertInputsPCA(X_train2, X_test2, MFprints, n_comp=10, type = 'All')

# ML models

## Support Vector Regression (SVR)

### Activity

In [21]:
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.variable import Real, Integer, Choice, Binary

#### using just SVR without tunning the paramiters and using A0 input

In [22]:
model1 = SVR()
model1.fit(X_train1, y_train1)
r_score = model1.score(X_test1, y_test1)
r_score

0.601330574641226

Clearly, the accuracy is very low so that is why they decided to tune the hyperparameters of SVM using Pyomo library

#### Def the the three classes to look the best hyperparamiters (using pymoo) 
- Class SVR model
- Class SVR solver
- Class Trainers.trainSVR_SingleOutput

For optimization using pymoo they are three things that needs to be define:
 - The problem: In this ElemetwiseProblem was use. (they are 2 more, problem and FunctionalProblem) it needs to be define as class.
 - The algorithm: In this case she use  NSGA2 or NSGA-II (Nondominated Sorting Genetic Algorithm II) is a popular multi-objective optimization algorithm used to solve optimization problems involving multiple conflicting objectives. 
 - Minimization: To optain an optimal solution

##### Class SVR Model

This class defines an optimization problem for SVR hyperparameter tuning using the ElementwiseProblem class from pymoo.

The two hyperparameter use are C and epsilon:
- C: Regularization parameter. It controls the trade-off between achieving a low training error and a low testing error. Higher values of C allow for more complex decision boundaries that fit the training data closely.
- Epsilon (ε): The margin of tolerance where no penalty is given to errors within this range. It defines the width of the epsilon-tube where errors are ignored.

Understanding the hyperpameter used

In [22]:
#She decided to store 2 of the 4 hyperparameter for SVR. She made them real and specifies the range of valid values for the hyperparameter.
#vars = {
            "C": Real(bounds = (0.1, 100)),
            "epsilon": Real(bounds = (0.01, 20.0))
        }

In [23]:
#vars['C']

The _evaluate method is a required method in classes that inherit from ElementwiseProblem in the pymoo library. It defines how to evaluate the objective functions for a given set of hyperparameters.

In [23]:
class svrModel(ElementwiseProblem):
    def __init__(self, X_train, X_test, y_train, y_test, **kwargs):
        vars = {
            "C": Real(bounds = (0.1, 100)),
            "epsilon": Real(bounds = (0.01, 20.0))
        }

        super().__init__(vars=vars, n_obj=3, n_ieq_constr=0, **kwargs)

        self.trainer = Trainers()

        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test

#The _evaluate method is a required method in classes that inherit from ElementwiseProblem in the pymoo library. It defines how to evaluate the objective functions for a given set of hyperparameters.
#The function values are supposed to be written into out["F"] and the constraints into out["G"] if n_constr is greater than zero.
    def _evaluate(self, X, out, *arg, **kwargs):

        scores = self.trainer.trainSVR_SingleOutput(h_params= X,
                                       X_train=self.X_train,
                                       X_test=self.X_test,
                                       y_train=self.y_train,
                                       y_test=self.y_test,
                                       folds=5)

        out['F'] = [-1 * scores[0], scores[1], -1 * scores[2]]

##### Class SVR Solvers

In this class it will be define:
- the algorithm: NSGA2
- the minimization: that add the problem, algorithm and the termination 

In [24]:
class rfrSolvers(ElementwiseProblem):
    def __init__(self):
        super(ElementwiseProblem, self).__init__()

    def rfrSolver(self, X_train, X_test, y_train, y_test):

        problem = rfrModel(X_train, X_test, y_train, y_test)

        #change the pop-size, n_offsprings, n_gen (if needed)
        algorithm = NSGA2(  
                        pop_size=2,
                        sampling=MixedVariableSampling(),
                        mating=MixedVariableMating(eliminate_duplicates=MixedVariableDuplicateElimination()),
                        eliminate_duplicates=MixedVariableDuplicateElimination(),
                        n_offsprings=2
                        )


        res = minimize(problem,
                       algorithm,
                       termination=("n_gen", 200),
                       seed=1,
                       save_history=True,
                       verbose=True)

        return res

    def get_best(self, res):

        for i in range(len(res.F)):

            w_metric = -1 * res.F[:, 0] - res.F[:, 1] - res.F[:, 2]

            if w_metric[i] == max(w_metric[:]):
                best_x = res.X[i]
                best_f = res.F[i]

        return best_x, best_f

##### Class Trainers.trainSVR_SingleOutput

This code defines a Trainers class that includes methods for training and evaluating an SVR model using different hyperparameters

In [25]:
class Trainers():
    def __init__(self):
        pass


    def trainSVR_SingleOutput(self, h_params, X_train, X_test, y_test, y_train, folds=5):
        """
        It takes in hyperparameters, training and testing data, and returns the mean and standard
        deviation of the cross validation scores, and the R^2 score of the model.
        
        :param h_params: hyperparameters
        :param X_train: the training data
        :param X_test: the test data
        :param y_test: the actual values of the target variable
        :param y_train: the training data
        :param folds: number of folds for cross validation, defaults to 5 (optional)
        :return: The mean and standard deviation of the cross validation scores, and the r_score.
        """
        print(h_params)

        kernel = 'rbf'
        C = h_params['C']
        #gamma = 'auto'
        epsilon = h_params['epsilon']

        #model = SVR(kernel=kernel, C=C, gamma=gamma, epsilon=epsilon)
        model = SVR(kernel=kernel, C=C, epsilon=epsilon)

        if folds is not None:
            scores = cross_val_score(model, X_train, y_train, cv=folds)

            model.fit(X_train, y_train)

            r_score = model.score(X_test, y_test)

            return scores.mean(), scores.std(), r_score

        else:

            model.fit(X_train, y_train)

            return model

Testing the code for A0

In [26]:
solver = svrSolvers()
trainer = Trainers()

res_A0 = solver.svrSolver(X_train=X_train1,
                X_test=X_test1,
                y_train=y_train1,
                y_test=y_test1)

h_params_A0, best_F_A0 = solver.get_best(res_A0)

score = trainer.trainSVR_SingleOutput(h_params=h_params_A0,
                        X_train=X_train1,
                        X_test=X_test1,
                        y_train=y_train1,
                        y_test=y_test1,
                        folds=5)

{'C': 41.76049826978715, 'epsilon': 0.012286352598724284}
{'C': 72.0604168948716, 'epsilon': 6.053628126910477}
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        2 |      1 |             - |             -
{'C': 72.0604168948716, 'epsilon': 0.3009392332060581}
{'C': 41.76049826978715, 'epsilon': 6.069687443092774}
{'C': 52.864167896745705, 'epsilon': 5.949025304005646}
{'C': 72.0604168948716, 'epsilon': 0.5072755760661412}
     2 |        6 |      1 |  0.000000E+00 |             f
{'C': 41.76049826978715, 'epsilon': 1.7926026381996865}
{'C': 66.10316060065897, 'epsilon': 0.3009392332060581}
{'C': 72.0604168948716, 'epsilon': 0.0298703801962584}
{'C': 78.65931170889557, 'epsilon': 0.013811636386431708}
     3 |       10 |      1 |  0.0226168543 |         ideal
{'C': 71.62019013087945, 'epsilon': 1.1894834012560185}
{'C': 84.36311326889772, 'epsilon': 0.015036048508410867}
{'C': 73.19186516411669, 'epsilon': 1.8941337826484623}
{'C': 71.4474218719919, 'epsilon': 1

In [27]:
score[0]

0.9478227604870295

In [28]:
score[1]

0.03114695063775072

In [29]:
score[2]

0.8695343986496258

In [30]:
print(h_params_A0)
print("%0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

{'C': 95.98983019144184, 'epsilon': 0.021496048288544144}
0.95 accuracy with a standard deviation of 0.03. R^2: 0.87


Testing the code for A1

In [31]:
solver = svrSolvers()
trainer = Trainers()

res_A1 = solver.svrSolver(X_train=pca_X1_train_MDFP,
                X_test=pca_X1_test_MDFP,
                y_train=y_train1,
                y_test=y_test1)

h_params_A1, best_F_A1 = solver.get_best(res_A1)

score = trainer.trainSVR_SingleOutput(h_params=h_params_A1,
                        X_train=pca_X1_train_MDFP,
                        X_test=pca_X1_test_MDFP,
                        y_train=y_train1,
                        y_test=y_test1,
                        folds=5)

{'C': 41.76049826978715, 'epsilon': 0.012286352598724284}
{'C': 72.0604168948716, 'epsilon': 6.053628126910477}
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        2 |      1 |             - |             -
{'C': 72.0604168948716, 'epsilon': 0.3009392332060581}
{'C': 41.76049826978715, 'epsilon': 6.069687443092774}
{'C': 52.864167896745705, 'epsilon': 5.949025304005646}
{'C': 72.0604168948716, 'epsilon': 0.5072755760661412}
     2 |        6 |      1 |  0.000000E+00 |             f
{'C': 41.76049826978715, 'epsilon': 1.7926026381996865}
{'C': 66.10316060065897, 'epsilon': 0.3009392332060581}
{'C': 72.0604168948716, 'epsilon': 0.0298703801962584}
{'C': 78.65931170889557, 'epsilon': 0.013811636386431708}
     3 |       10 |      1 |  0.0190959407 |         ideal
{'C': 71.62019013087945, 'epsilon': 1.1894834012560185}
{'C': 84.36311326889772, 'epsilon': 0.015036048508410867}
{'C': 73.19186516411669, 'epsilon': 1.8941337826484623}
{'C': 71.4474218719919, 'epsilon': 1

In [32]:
print(h_params_A1)
print("%0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

{'C': 99.99998334842246, 'epsilon': 0.022722554451559643}
0.95 accuracy with a standard deviation of 0.03. R^2: 0.87


Testing the code for A2

In [33]:
solver = svrSolvers()
trainer = Trainers()

res_A2 = solver.svrSolver(X_train=pca_X1_train_ALL,
                X_test=pca_X1_test_ALL,
                y_train=y_train1,
                y_test=y_test1)

h_params_A2, best_F_A2 = solver.get_best(res_A2)
# I change X_train and X_test for pca_x1_train_all to see if there is any diffence but still not see the diffence
score = trainer.trainSVR_SingleOutput(h_params=h_params_A2,
                        X_train=pca_X1_train_ALL,
                        X_test=pca_X1_test_ALL,
                        y_train=y_train1,
                        y_test=y_test1,
                        folds=5)

{'C': 41.76049826978715, 'epsilon': 0.012286352598724284}
{'C': 72.0604168948716, 'epsilon': 6.053628126910477}
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        2 |      1 |             - |             -
{'C': 72.0604168948716, 'epsilon': 0.3009392332060581}
{'C': 41.76049826978715, 'epsilon': 6.069687443092774}
{'C': 52.864167896745705, 'epsilon': 5.949025304005646}
{'C': 72.0604168948716, 'epsilon': 0.5072755760661412}
     2 |        6 |      1 |  0.000000E+00 |             f
{'C': 41.76049826978715, 'epsilon': 1.7926026381996865}
{'C': 66.10316060065897, 'epsilon': 0.3009392332060581}
{'C': 72.0604168948716, 'epsilon': 0.0298703801962584}
{'C': 78.65931170889557, 'epsilon': 0.013811636386431708}
     3 |       10 |      1 |  0.0309778916 |         ideal
{'C': 72.0604168948716, 'epsilon': 0.014091181742050613}
{'C': 76.22069036392507, 'epsilon': 0.02940404251381104}
{'C': 75.0676412559542, 'epsilon': 2.457827844596563}
{'C': 69.17206279904605, 'epsilon': 0.

In [34]:
print(h_params_A2)
print("%0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

{'C': 99.87571284612757, 'epsilon': 0.023340201888221765}
0.95 accuracy with a standard deviation of 0.03. R^2: 0.84


## Artificial Neural Network (ANN)

In [38]:
from src.trainers import Trainers

In [39]:
solver = annSolvers()
trainer = Trainers()

res = solver.annSolver(X_train=pca_X1_train_ALL,
                X_test=pca_X1_test_ALL,
                y_train=y_train1,
                y_test=y_test1)

h_params, best_F = solver.get_best(res)

score = trainer.trainANN_SingleOutput(h_params=h_params,
                        X_train=X_train1,
                        X_test=X_test1,
                        y_train=y_train1,
                        y_test=y_test1,
                        folds=5)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        2 |      2 |             - |             -
     2 |        6 |      2 |  1.1456993160 |         ideal
     3 |       10 |      2 |  1.448055E+01 |         ideal
     4 |       14 |      2 |  0.000000E+00 |             f
     5 |       18 |      2 |  0.000000E+00 |             f
     6 |       22 |      2 |  0.5122822172 |         ideal
     7 |       26 |      2 |  1.2358052832 |         ideal
     8 |       30 |      2 |  2.7746002876 |         ideal
     9 |       34 |      2 |  1.0000000000 |         ideal
    10 |       38 |      2 |  0.000000E+00 |             f


In [40]:
print(h_params)
print("%0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

activation                 = ['logistic', 'tanh', 'relu']
solver                     = ['lbfgs', 'adam']
learning_rate              = ['constant', 'invscaling', 'adaptive']

{'alpha': 0.1, 'learning_rate_init': 0.1, 'activation_idx': 1, 'solver_idx': 1, 'learning_rate_idx': 0}
0.28 accuracy with a standard deviation of 0.81. R^2: 0.82


## Random Forest Regression (RFR)

In [42]:
solver = rfrSolvers()
trainer = Trainers()


res = solver.rfrSolver(X_train=pca_X1_train_ALL,
                X_test=pca_X1_test_ALL,
                y_train=y_train1,
                y_test=y_test1)

h_params, best_F = solver.get_best(res)

score = trainer.trainRFR_SingleOutput(h_params=h_params,
                        X_train=X_train1,
                        X_test=X_test1,
                        y_train=y_train1,
                        y_test=y_test1,
                        folds=5)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |        2 |      1 |             - |             -
     2 |        4 |      2 |  1.0000000000 |         ideal
     3 |        6 |      2 |  1.4587597603 |         ideal
     4 |        8 |      2 |  2.6210143489 |         ideal
     5 |       10 |      2 |  3.549368E+02 |         ideal
     6 |       12 |      2 |  1.0000000000 |         ideal
     7 |       14 |      2 |  0.000000E+00 |             f
     8 |       16 |      2 |  0.000000E+00 |             f
     9 |       18 |      2 |  1.0000000000 |         ideal
    10 |       20 |      2 |  1.0000000000 |         ideal
    11 |       22 |      2 |  1.1048367680 |         ideal
    12 |       24 |      2 |  0.6455036909 |         ideal
    13 |       26 |      2 |  0.000000E+00 |             f
    14 |       28 |      2 |  0.000000E+00 |             f
    15 |       30 |      2 |  0.2430815761 |         ideal
    16 |       32 |      2 |  0.000000E+00 |            

In [43]:
max_features = ['auto', 'sqrt','log2']
bootstrap = ['True', 'False']
print(h_params)
print("%0.2f accuracy with a standard deviation of %0.2f. R^2: %0.2f" %
        (score[0], score[1], score[2]))

{'max_depth': 59, 'n_estimators': 44, 'max_features_idx': 1, 'bootstrap_idx': 1, 'min_weight_fraction_leaf': 0.00016097904672728605}
0.93 accuracy with a standard deviation of 0.04. R^2: 0.70
